In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import avg, length
import time

spark = SparkSession.builder \
    .master("spark://192.168.2.62:7077") \
    .appName("YourAppName") \
    .config("spark.executor.memory","750m") \
    .getOrCreate()

   
sc = spark.sparkContext

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/03/13 11:15:25 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
data_path = "hdfs://192.168.2.62:9001/user/root/data/processed_objects.json"
df = spark.read.json(data_path)
#df = df.limit(100000)
#df.rdd.count()

24/03/13 11:15:47 WARN TaskSchedulerImpl: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
24/03/13 11:16:02 WARN TaskSchedulerImpl: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
24/03/13 11:16:17 WARN TaskSchedulerImpl: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
24/03/13 11:16:32 WARN TaskSchedulerImpl: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources


In [4]:
df = df.limit(125000)
df.rdd.count()

125000

In [4]:
for _ in range(10):
    start_time = time.time()
    df_time = df.withColumn("content_length", length("content"))
    df_time = df_time.withColumn("summary_length", length("summary"))
    # Group by subreddit and calculate average lengths
    average_lengths = df_time.groupBy("subreddit").agg(
        avg("content_length").alias("avg_content_length"),
        avg("summary_length").alias("avg_summary_length")
    )
    average_lengths = average_lengths.orderBy("subreddit")
    average_lengths.rdd.count()
    end_time = time.time()
    print(end_time-start_time)

29.48692297935486


13.582839727401733


10.528473854064941


18.88355565071106


24.3319091796875


8.957468271255493


9.77723741531372


11.038676977157593


13.566510438919067


12.896420240402222


In [17]:
from pyspark.sql.functions import col
ratio_df = average_lengths.withColumn("ratio", col("avg_content_length") / col("avg_summary_length"))

# Sort the DataFrame by the ratio column in ascending order
sorted_df = ratio_df.orderBy(desc("ratio"))
sorted_df.show()

+--------------------+------------------+------------------+------------------+
|           subreddit|avg_content_length|avg_summary_length|             ratio|
+--------------------+------------------+------------------+------------------+
|  MakeNewFriendsHere|            6442.0|               5.0|            1288.4|
|    AnythingGoesPics|            2518.0|               4.0|             629.5|
|     LiveFromNewYork|            3728.0|               6.0| 621.3333333333334|
|           TEOTWAWKI|            2759.0|               5.0|             551.8|
|            RPGMaker|            5332.0|              10.0|             533.2|
|      YellowcardRock|            5833.0|              11.0| 530.2727272727273|
|       VeryExpensive|            4108.0|               9.0|456.44444444444446|
|                 hsp|            7446.0|              18.0| 413.6666666666667|
|            maxpayne|            2723.0|               8.0|           340.375|
|           terrorism|            3830.0

In [10]:
from pyspark.sql.functions import desc
average_lengths = average_lengths.orderBy(desc("avg_content_length"))
average_lengths.show()

+--------------------+------------------+------------------+
|           subreddit|avg_content_length|avg_summary_length|
+--------------------+------------------+------------------+
|            coredump|            9716.0|              31.0|
|RedditsMuseumofFilth|            9388.0|              37.0|
|  AtheistToChristian|            8286.0|              45.0|
|        JurassicPark|            8176.0|             526.0|
|            askscifi|            8104.0|              77.0|
|         bostontrees|            7831.0|              85.0|
|            Arkansas|            7715.0|             446.0|
|        TIRNTASPHUCK|            7479.0|              39.0|
|                 hsp|            7446.0|              18.0|
|  parentingthegifted|            7302.0|             201.0|
|        knowyourshit|            7156.0|              96.0|
|         GetEmployed|            7034.0|              24.0|
|      backwardsmusic|            7003.0|             549.0|
|     northernireland|  

In [5]:
sc.stop()

24/03/12 16:41:54 ERROR TransportRequestHandler: Error while invoking RpcHandler#receive() for one-way message.
org.apache.spark.SparkException: Could not find AppClient.
	at org.apache.spark.rpc.netty.Dispatcher.postMessage(Dispatcher.scala:178)
	at org.apache.spark.rpc.netty.Dispatcher.postOneWayMessage(Dispatcher.scala:150)
	at org.apache.spark.rpc.netty.NettyRpcHandler.receive(NettyRpcEnv.scala:690)
	at org.apache.spark.network.server.TransportRequestHandler.processOneWayMessage(TransportRequestHandler.java:274)
	at org.apache.spark.network.server.TransportRequestHandler.handle(TransportRequestHandler.java:111)
	at org.apache.spark.network.server.TransportChannelHandler.channelRead0(TransportChannelHandler.java:140)
	at org.apache.spark.network.server.TransportChannelHandler.channelRead0(TransportChannelHandler.java:53)
	at io.netty.channel.SimpleChannelInboundHandler.channelRead(SimpleChannelInboundHandler.java:99)
	at io.netty.channel.AbstractChannelHandlerContext.invokeChannelRe

In [ ]:
# Order by subreddit for readability (optional, can be removed for large datasets)

# Show the results
#average_lengths.show()
# Optionally, save the results to a file
#output_path = "/home/ubuntu/Drive/reddit_data/output/average_lengths.csv"
#average_lengths.write.csv(output_path, header=True)